In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.utils import shuffle
from sklearn import cross_validation

C:\ProgramData\Anaconda2\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
travel_time_train = pd.read_table('./preprocessed_data/travel_time_train.txt', sep=';')
prediction_table = pd.read_table('./preprocessed_data/prediction_table.txt', sep=';')

In [3]:
prediction_table.head()

,link_ID,date,time_interval,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes,interval_num,mean_last_1hour,median_last_1hour,...,width_std,in_links_num_std,out_links_num_std,travel_time_7days_before,travel_time_15days_before,mean_all,median_all,std_all,mode_all,travel_time
0,3377906280028510514_,2017-07-01,"[2017-07-01 08:00:00,2017-07-01 08:02:00)",2017-07-01 08:00:00,2017-07-01 08:02:00,8,0,0.0,5.742857,5.4,...,0.0,0.25,0.25,18.8,4.4,6.195238,5.40,2.018808,5.0,0
1,3377906280028510514_,2017-07-01,"[2017-07-01 08:02:00,2017-07-01 08:04:00)",2017-07-01 08:02:00,2017-07-01 08:04:00,8,2,1.0,5.742857,5.4,...,0.0,0.25,0.25,18.8,4.4,6.628205,5.70,2.902082,4.8,0
2,3377906280028510514_,2017-07-01,"[2017-07-01 08:04:00,2017-07-01 08:06:00)",2017-07-01 08:04:00,2017-07-01 08:06:00,8,4,2.0,5.742857,5.4,...,0.0,0.25,0.25,13.6,4.4,5.944186,5.40,2.180575,4.9,0
3,3377906280028510514_,2017-07-01,"[2017-07-01 08:06:00,2017-07-01 08:08:00)",2017-07-01 08:06:00,2017-07-01 08:08:00,8,6,3.0,5.742857,5.4,...,0.0,0.25,0.25,13.2,4.4,6.100000,5.70,2.270162,6.0,0
4,3377906280028510514_,2017-07-01,"[2017-07-01 08:08:00,2017-07-01 08:10:00)",2017-07-01 08:08:00,2017-07-01 08:10:00,8,8,4.0,5.742857,5.4,...,0.0,0.25,0.25,15.3,4.4,6.540476,5.55,3.196802,4.8,0


In [4]:
travel_time_train_shuffle = shuffle(travel_time_train)

In [5]:
p = 0.8
train_set = travel_time_train_shuffle[:int(len(travel_time_train_shuffle)*p)]
test_set = travel_time_train_shuffle[int(len(travel_time_train_shuffle)*p):]

In [6]:
prediction_table.dtypes

link_ID                       object
date                          object
time_interval                 object
time_interval_begin           object
time_interval_end             object
time_interval_hour             int64
time_interval_minutes          int64
interval_num                 float64
mean_last_1hour              float64
median_last_1hour            float64
std_last_1hour               float64
mode_last_1hour              float64
max_last_1hour               float64
min_last_1hour               float64
mean_last_half_hour          float64
median_last_half_hour        float64
std_last_half_hour           float64
mode_last_half_hour          float64
max_last_half_hour           float64
min_last_half_hour           float64
time_stage1                  float64
time_stage2                  float64
time_stage3                  float64
interval_num_1               float64
interval_num_2               float64
interval_num_3               float64
interval_num_4               float64
i

In [7]:
train_set.head()

,link_ID,date,time_interval,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes,interval_num,mean_last_1hour,median_last_1hour,...,width_std,in_links_num_std,out_links_num_std,travel_time_7days_before,travel_time_15days_before,mean_all,median_all,std_all,mode_all,travel_time
116860,4377906284514600514__,2017-06-05 00:00:00,"[2017-06-05 08:06:00,2017-06-05 08:08:00)",2017-06-05 08:06:00,2017-06-05 08:08:00,8,6,3.0,9.036667,7.90,...,0.50,0.50,0.25,9.0,7.0,8.459091,7.80,2.338662,6.7,7.4
231939,4377906289663800514__,2017-06-14 00:00:00,"[2017-06-14 08:28:00,2017-06-14 08:30:00)",2017-06-14 08:28:00,2017-06-14 08:30:00,8,28,14.0,32.806897,16.20,...,0.75,0.50,0.25,34.6,10.2,19.007955,11.55,16.186401,5.7,42.2
436933,4377906287063800514__,2017-06-08 00:00:00,"[2017-06-08 15:56:00,2017-06-08 15:58:00)",2017-06-08 15:56:00,2017-06-08 15:58:00,15,56,28.0,21.150000,19.40,...,0.50,0.25,0.25,20.4,17.6,16.968132,16.40,6.450286,10.2,18.0
632645,4377906282959500514__,2017-05-24 00:00:00,"[2017-05-24 18:56:00,2017-05-24 18:58:00)",2017-05-24 18:56:00,2017-05-24 18:58:00,18,56,28.0,43.028571,44.45,...,0.75,0.25,0.50,19.7,20.8,22.285882,20.80,11.856705,5.4,26.7
72766,4377906281784800514__,2017-05-17 00:00:00,"[2017-05-17 08:36:00,2017-05-17 08:38:00)",2017-05-17 08:36:00,2017-05-17 08:38:00,8,36,18.0,9.896667,9.40,...,0.50,0.25,0.25,10.7,13.2,11.642697,10.70,4.186067,9.3,15.2


In [8]:
X_train = train_set.iloc[:, 8:-1] #np.array()
Y_train = train_set.iloc[:, -1] #np.array()
X_test = test_set.iloc[:, 8:-1] #np.array()
Y_test = test_set.iloc[:, -1] #np.array()

In [9]:
X_train.shape

(646527, 62)

In [10]:
X_train.dtypes

mean_last_1hour              float64
median_last_1hour            float64
std_last_1hour               float64
mode_last_1hour              float64
max_last_1hour               float64
min_last_1hour               float64
mean_last_half_hour          float64
median_last_half_hour        float64
std_last_half_hour           float64
mode_last_half_hour          float64
max_last_half_hour           float64
min_last_half_hour           float64
time_stage1                  float64
time_stage2                  float64
time_stage3                  float64
interval_num_1               float64
interval_num_2               float64
interval_num_3               float64
interval_num_4               float64
interval_num_5               float64
interval_num_6               float64
interval_num_7               float64
interval_num_8               float64
interval_num_9               float64
interval_num_10              float64
interval_num_11              float64
interval_num_12              float64
i

In [11]:
print(len(X_train))
print(len(Y_train))

646527
646527


In [12]:
def mape_object(y,d):
    g=1.0*np.sign(y-d)/d
    h=1.0/d
    return -g,h

In [13]:
clf = xgb.XGBRegressor(max_depth=12,
                       learning_rate=0.05,
                       n_estimators=3000,
                       silent=True,
                       objective=mape_object,
                       gamma=0,
                       min_child_weight=5,
                       max_delta_step=0,
                       subsample=0.8,
                       colsample_bytree=0.8,
                       colsample_bylevel=1,
                       reg_alpha=0.75,
                       reg_lambda=0.75,
                       scale_pos_weight=1,
                       seed=0,
                       missing=None)


clf.fit(X_train, Y_train, eval_metric='rmse', verbose = True, eval_set = [(X_test, Y_test)],early_stopping_rounds=1000)

Will train until validation_0 error hasn't decreased in 100 rounds.
[0]	validation_0-rmse:34.125458
[1]	validation_0-rmse:34.097187
[2]	validation_0-rmse:34.068962
[3]	validation_0-rmse:34.040787
[4]	validation_0-rmse:34.012657
[5]	validation_0-rmse:33.984592
[6]	validation_0-rmse:33.956570
[7]	validation_0-rmse:33.928600
[8]	validation_0-rmse:33.900688
[9]	validation_0-rmse:33.872814
[10]	validation_0-rmse:33.844990
[11]	validation_0-rmse:33.817230
[12]	validation_0-rmse:33.789520
[13]	validation_0-rmse:33.761864
[14]	validation_0-rmse:33.734261
[15]	validation_0-rmse:33.706707
[16]	validation_0-rmse:33.679207
[17]	validation_0-rmse:33.651768
[18]	validation_0-rmse:33.624374
[19]	validation_0-rmse:33.597038
[20]	validation_0-rmse:33.569748
[21]	validation_0-rmse:33.542515
[22]	validation_0-rmse:33.515331
[23]	validation_0-rmse:33.488205
[24]	validation_0-rmse:33.461124
[25]	validation_0-rmse:33.434105
[26]	validation_0-rmse:33.407139
[27]	validation_0-rmse:33.380226
[28]	validation_0-

[243]	validation_0-rmse:28.676651
[244]	validation_0-rmse:28.659382
[245]	validation_0-rmse:28.642179
[246]	validation_0-rmse:28.624954
[247]	validation_0-rmse:28.607788
[248]	validation_0-rmse:28.590675
[249]	validation_0-rmse:28.573591
[250]	validation_0-rmse:28.556559
[251]	validation_0-rmse:28.539541
[252]	validation_0-rmse:28.522549
[253]	validation_0-rmse:28.505590
[254]	validation_0-rmse:28.488697
[255]	validation_0-rmse:28.471790
[256]	validation_0-rmse:28.454922
[257]	validation_0-rmse:28.438055
[258]	validation_0-rmse:28.421305
[259]	validation_0-rmse:28.404530
[260]	validation_0-rmse:28.387791
[261]	validation_0-rmse:28.371077
[262]	validation_0-rmse:28.354448
[263]	validation_0-rmse:28.337799
[264]	validation_0-rmse:28.321175
[265]	validation_0-rmse:28.304600
[266]	validation_0-rmse:28.288063
[267]	validation_0-rmse:28.271549
[268]	validation_0-rmse:28.255117
[269]	validation_0-rmse:28.238684
[270]	validation_0-rmse:28.222338
[271]	validation_0-rmse:28.206038
[272]	validati

[484]	validation_0-rmse:25.382011
[485]	validation_0-rmse:25.371450
[486]	validation_0-rmse:25.360952
[487]	validation_0-rmse:25.350431
[488]	validation_0-rmse:25.339979
[489]	validation_0-rmse:25.329563
[490]	validation_0-rmse:25.319080
[491]	validation_0-rmse:25.308668
[492]	validation_0-rmse:25.298243
[493]	validation_0-rmse:25.287838
[494]	validation_0-rmse:25.277519
[495]	validation_0-rmse:25.267193
[496]	validation_0-rmse:25.256912
[497]	validation_0-rmse:25.246666
[498]	validation_0-rmse:25.236420
[499]	validation_0-rmse:25.226179
[500]	validation_0-rmse:25.215944
[501]	validation_0-rmse:25.205769
[502]	validation_0-rmse:25.195660
[503]	validation_0-rmse:25.185402
[504]	validation_0-rmse:25.175449
[505]	validation_0-rmse:25.165380
[506]	validation_0-rmse:25.155228
[507]	validation_0-rmse:25.145153
[508]	validation_0-rmse:25.135067
[509]	validation_0-rmse:25.124994
[510]	validation_0-rmse:25.114943
[511]	validation_0-rmse:25.105003
[512]	validation_0-rmse:25.095064
[513]	validati

[725]	validation_0-rmse:23.347836
[726]	validation_0-rmse:23.341127
[727]	validation_0-rmse:23.334429
[728]	validation_0-rmse:23.327682
[729]	validation_0-rmse:23.320942
[730]	validation_0-rmse:23.314190
[731]	validation_0-rmse:23.307497
[732]	validation_0-rmse:23.300858
[733]	validation_0-rmse:23.294191
[734]	validation_0-rmse:23.287453
[735]	validation_0-rmse:23.280817
[736]	validation_0-rmse:23.274082
[737]	validation_0-rmse:23.267427
[738]	validation_0-rmse:23.260771
[739]	validation_0-rmse:23.254208
[740]	validation_0-rmse:23.247683
[741]	validation_0-rmse:23.241030
[742]	validation_0-rmse:23.234505
[743]	validation_0-rmse:23.227959
[744]	validation_0-rmse:23.221289
[745]	validation_0-rmse:23.214684
[746]	validation_0-rmse:23.208189
[747]	validation_0-rmse:23.201744
[748]	validation_0-rmse:23.195234
[749]	validation_0-rmse:23.188730
[750]	validation_0-rmse:23.182238
[751]	validation_0-rmse:23.175674
[752]	validation_0-rmse:23.169285
[753]	validation_0-rmse:23.162800
[754]	validati

[966]	validation_0-rmse:21.981903
[967]	validation_0-rmse:21.977205
[968]	validation_0-rmse:21.972523
[969]	validation_0-rmse:21.967667
[970]	validation_0-rmse:21.962999
[971]	validation_0-rmse:21.958206
[972]	validation_0-rmse:21.953495
[973]	validation_0-rmse:21.948805
[974]	validation_0-rmse:21.944103
[975]	validation_0-rmse:21.939413
[976]	validation_0-rmse:21.934755
[977]	validation_0-rmse:21.929941
[978]	validation_0-rmse:21.925280
[979]	validation_0-rmse:21.920650
[980]	validation_0-rmse:21.916054
[981]	validation_0-rmse:21.911444
[982]	validation_0-rmse:21.906816
[983]	validation_0-rmse:21.902052
[984]	validation_0-rmse:21.897457
[985]	validation_0-rmse:21.892767
[986]	validation_0-rmse:21.888229
[987]	validation_0-rmse:21.883575
[988]	validation_0-rmse:21.878998
[989]	validation_0-rmse:21.874371
[990]	validation_0-rmse:21.869682
[991]	validation_0-rmse:21.865099
[992]	validation_0-rmse:21.860582
[993]	validation_0-rmse:21.856037
[994]	validation_0-rmse:21.851559
[995]	validati

...

[2142]	validation_0-rmse:19.337931
[2143]	validation_0-rmse:19.337160
[2144]	validation_0-rmse:19.336374
[2145]	validation_0-rmse:19.335703
[2146]	validation_0-rmse:19.334858
[2147]	validation_0-rmse:19.334272
[2148]	validation_0-rmse:19.333509
[2149]	validation_0-rmse:19.332775
[2150]	validation_0-rmse:19.331938
[2151]	validation_0-rmse:19.331131
[2152]	validation_0-rmse:19.330336
[2153]	validation_0-rmse:19.329636
[2154]	validation_0-rmse:19.328918
[2155]	validation_0-rmse:19.328070
[2156]	validation_0-rmse:19.327484
[2157]	validation_0-rmse:19.326698
[2158]	validation_0-rmse:19.325979
[2159]	validation_0-rmse:19.325268
[2160]	validation_0-rmse:19.324644
[2161]	validation_0-rmse:19.323824
[2162]	validation_0-rmse:19.323000
[2163]	validation_0-rmse:19.322222
[2164]	validation_0-rmse:19.321442
[2165]	validation_0-rmse:19.320707
[2166]	validation_0-rmse:19.319914
[2167]	validation_0-rmse:19.319279
[2168]	validation_0-rmse:19.318375
[2169]	validation_0-rmse:19.317595
[2170]	validation_0-

[2377]	validation_0-rmse:19.195633
[2378]	validation_0-rmse:19.195227
[2379]	validation_0-rmse:19.194714
[2380]	validation_0-rmse:19.194378
[2381]	validation_0-rmse:19.193909
[2382]	validation_0-rmse:19.193413
[2383]	validation_0-rmse:19.193005
[2384]	validation_0-rmse:19.192373
[2385]	validation_0-rmse:19.191833
[2386]	validation_0-rmse:19.191366
[2387]	validation_0-rmse:19.190969
[2388]	validation_0-rmse:19.190435
[2389]	validation_0-rmse:19.189863
[2390]	validation_0-rmse:19.189453
[2391]	validation_0-rmse:19.189066
[2392]	validation_0-rmse:19.188610
[2393]	validation_0-rmse:19.188139
[2394]	validation_0-rmse:19.187643
[2395]	validation_0-rmse:19.187222
[2396]	validation_0-rmse:19.186769
[2397]	validation_0-rmse:19.186268
[2398]	validation_0-rmse:19.185776
[2399]	validation_0-rmse:19.185421
[2400]	validation_0-rmse:19.184811
[2401]	validation_0-rmse:19.184343
[2402]	validation_0-rmse:19.183949
[2403]	validation_0-rmse:19.183533
[2404]	validation_0-rmse:19.183029
[2405]	validation_0-

[2612]	validation_0-rmse:19.100927
[2613]	validation_0-rmse:19.100658
[2614]	validation_0-rmse:19.100315
[2615]	validation_0-rmse:19.099943
[2616]	validation_0-rmse:19.099554
[2617]	validation_0-rmse:19.099209
[2618]	validation_0-rmse:19.098894
[2619]	validation_0-rmse:19.098518
[2620]	validation_0-rmse:19.098103
[2621]	validation_0-rmse:19.097660
[2622]	validation_0-rmse:19.097292
[2623]	validation_0-rmse:19.096909
[2624]	validation_0-rmse:19.096571
[2625]	validation_0-rmse:19.096237
[2626]	validation_0-rmse:19.095894
[2627]	validation_0-rmse:19.095598
[2628]	validation_0-rmse:19.095196
[2629]	validation_0-rmse:19.094845
[2630]	validation_0-rmse:19.094501
[2631]	validation_0-rmse:19.094131
[2632]	validation_0-rmse:19.093874
[2633]	validation_0-rmse:19.093569
[2634]	validation_0-rmse:19.093161
[2635]	validation_0-rmse:19.092768
[2636]	validation_0-rmse:19.092398
[2637]	validation_0-rmse:19.092039
[2638]	validation_0-rmse:19.091642
[2639]	validation_0-rmse:19.091341
[2640]	validation_0-

[2847]	validation_0-rmse:19.022902
[2848]	validation_0-rmse:19.022587
[2849]	validation_0-rmse:19.022299
[2850]	validation_0-rmse:19.022093
[2851]	validation_0-rmse:19.021738
[2852]	validation_0-rmse:19.021534
[2853]	validation_0-rmse:19.021248
[2854]	validation_0-rmse:19.020973
[2855]	validation_0-rmse:19.020752
[2856]	validation_0-rmse:19.020599
[2857]	validation_0-rmse:19.020382
[2858]	validation_0-rmse:19.020182
[2859]	validation_0-rmse:19.019865
[2860]	validation_0-rmse:19.019613
[2861]	validation_0-rmse:19.019457
[2862]	validation_0-rmse:19.019203
[2863]	validation_0-rmse:19.018978
[2864]	validation_0-rmse:19.018684
[2865]	validation_0-rmse:19.018377
[2866]	validation_0-rmse:19.018190
[2867]	validation_0-rmse:19.017885
[2868]	validation_0-rmse:19.017664
[2869]	validation_0-rmse:19.017378
[2870]	validation_0-rmse:19.017122
[2871]	validation_0-rmse:19.016865
[2872]	validation_0-rmse:19.016586
[2873]	validation_0-rmse:19.016361
[2874]	validation_0-rmse:19.016378
[2875]	validation_0-

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.05, max_delta_step=0, max_depth=12,
       min_child_weight=5, missing=None, n_estimators=3000, nthread=-1,
       objective=<function mape_object at 0x000000000A1AC908>,
       reg_alpha=0.75, reg_lambda=0.75, scale_pos_weight=1, seed=0,
       silent=True, subsample=0.8)

In [14]:
predictions = clf.predict(prediction_table.iloc[:, 8:-1])

In [15]:
len(predictions)

368280

In [16]:
prediction_table['travel_time_xgb'] = predictions

In [17]:
prediction_table.head()

,link_ID,date,time_interval,time_interval_begin,time_interval_end,time_interval_hour,time_interval_minutes,interval_num,mean_last_1hour,median_last_1hour,...,in_links_num_std,out_links_num_std,travel_time_7days_before,travel_time_15days_before,mean_all,median_all,std_all,mode_all,travel_time,travel_time_xgb
0,3377906280028510514_,2017-07-01,"[2017-07-01 08:00:00,2017-07-01 08:02:00)",2017-07-01 08:00:00,2017-07-01 08:02:00,8,0,0.0,5.742857,5.4,...,0.25,0.25,18.8,4.4,6.195238,5.40,2.018808,5.0,0,5.389344
1,3377906280028510514_,2017-07-01,"[2017-07-01 08:02:00,2017-07-01 08:04:00)",2017-07-01 08:02:00,2017-07-01 08:04:00,8,2,1.0,5.742857,5.4,...,0.25,0.25,18.8,4.4,6.628205,5.70,2.902082,4.8,0,5.602496
2,3377906280028510514_,2017-07-01,"[2017-07-01 08:04:00,2017-07-01 08:06:00)",2017-07-01 08:04:00,2017-07-01 08:06:00,8,4,2.0,5.742857,5.4,...,0.25,0.25,13.6,4.4,5.944186,5.40,2.180575,4.9,0,5.409872
3,3377906280028510514_,2017-07-01,"[2017-07-01 08:06:00,2017-07-01 08:08:00)",2017-07-01 08:06:00,2017-07-01 08:08:00,8,6,3.0,5.742857,5.4,...,0.25,0.25,13.2,4.4,6.100000,5.70,2.270162,6.0,0,5.433060
4,3377906280028510514_,2017-07-01,"[2017-07-01 08:08:00,2017-07-01 08:10:00)",2017-07-01 08:08:00,2017-07-01 08:10:00,8,8,4.0,5.742857,5.4,...,0.25,0.25,15.3,4.4,6.540476,5.55,3.196802,4.8,0,5.617159


In [18]:
final_predictions = prediction_table[['link_ID', 'date', 'time_interval', 'travel_time_xgb']]
final_predictions.head()

,link_ID,date,time_interval,travel_time_xgb
0,3377906280028510514_,2017-07-01,"[2017-07-01 08:00:00,2017-07-01 08:02:00)",5.389344
1,3377906280028510514_,2017-07-01,"[2017-07-01 08:02:00,2017-07-01 08:04:00)",5.602496
2,3377906280028510514_,2017-07-01,"[2017-07-01 08:04:00,2017-07-01 08:06:00)",5.409872
3,3377906280028510514_,2017-07-01,"[2017-07-01 08:06:00,2017-07-01 08:08:00)",5.433060
4,3377906280028510514_,2017-07-01,"[2017-07-01 08:08:00,2017-07-01 08:10:00)",5.617159


In [19]:
link_id = []
for index, row in final_predictions.iterrows():
    id_ = row[0][:-1]
    link_id.append(id_)
final_predictions['link_ID'] = link_id

C:\ProgramData\Anaconda2\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [20]:
final_predictions.head()

,link_ID,date,time_interval,travel_time_xgb
0,3377906280028510514,2017-07-01,"[2017-07-01 08:00:00,2017-07-01 08:02:00)",5.389344
1,3377906280028510514,2017-07-01,"[2017-07-01 08:02:00,2017-07-01 08:04:00)",5.602496
2,3377906280028510514,2017-07-01,"[2017-07-01 08:04:00,2017-07-01 08:06:00)",5.409872
3,3377906280028510514,2017-07-01,"[2017-07-01 08:06:00,2017-07-01 08:08:00)",5.433060
4,3377906280028510514,2017-07-01,"[2017-07-01 08:08:00,2017-07-01 08:10:00)",5.617159


In [21]:
final_predictions = final_predictions[['link_ID', 'date', 'time_interval', 'travel_time_xgb']]

In [22]:
final_predictions.head()

,link_ID,date,time_interval,travel_time_xgb
0,3377906280028510514,2017-07-01,"[2017-07-01 08:00:00,2017-07-01 08:02:00)",5.389344
1,3377906280028510514,2017-07-01,"[2017-07-01 08:02:00,2017-07-01 08:04:00)",5.602496
2,3377906280028510514,2017-07-01,"[2017-07-01 08:04:00,2017-07-01 08:06:00)",5.409872
3,3377906280028510514,2017-07-01,"[2017-07-01 08:06:00,2017-07-01 08:08:00)",5.433060
4,3377906280028510514,2017-07-01,"[2017-07-01 08:08:00,2017-07-01 08:10:00)",5.617159


In [23]:
final_predictions.isnull().any()

link_ID            False
date               False
time_interval      False
travel_time_xgb    False
dtype: bool

final_predictions.to_csv('./prediction_915_2/final_prediction_xgb.txt', sep=';', index=False)

In [24]:
final_predictions1 = final_predictions[:100000]
final_predictions2 = final_predictions[100000:200000]
final_predictions3 = final_predictions[200000:300000]
final_predictions4 = final_predictions[300000:]

final_predictions1.to_csv('./prediction_915_2/final_predictions_xgb1.txt', sep=';', index=False, encoding='utf-8') 
final_predictions2.to_csv('./prediction_915_2/final_predictions_xgb2.txt', sep=';', index=False, encoding='utf-8')
final_predictions3.to_csv('./prediction_915_2/final_predictions_xgb3.txt', sep=';', index=False, encoding='utf-8') 
final_predictions4.to_csv('./prediction_915_2/final_predictions_xgb4.txt', sep=';', index=False, encoding='utf-8')